# Setup

In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [2]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import math

home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

def SSH_connection(query):
  with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                            passwd=sql_password, db=sql_main_database,
                            port=tunnel.local_bind_port)
    print('connection done')
    df_result = pd.read_sql_query(query,conn)
  
  return df_result

In [10]:
# def remove_rows(df, desired_diff):
#     alt_df = df
#     # rows = alt_df.shape[0]
#     # for k in range(rows):
#     #     firstLogin = alt_df.loc[k].at['firstLogin']
#     #     lastLogin = alt_df.loc[k].at['lastLogin']
#     #     diff = lastLogin - firstLogin

#     #     if diff.days>desired_diff:
#     #         alt_df = alt_df.drop(k, 0)
    
#     for index, row in alt_df.iterrows():
#         diff = row['lastLogin'] - row['firstLogin']
#         if diff.days<desired_diff:
#             alt_df = alt_df.drop(index, 0)

#     return alt_df

In [4]:
# CALCULATING NUMBER OF USERS 
# query_users = ''' SELECT userId, firstLogin, lastLogin, DATEDIFF(lastLogin, firstLogin)+1 as diff from ML3_mirror.user_data WHERE EXISTS
#                   (SELECT userId from log_action WHERE tstamp>='2015-11-04' AND tstamp<'2016-04-01' AND log_action.userId = user_data.userId)
#                   AND firstLogin<'2015-12-01' '''
# users = SSH_connection(query_users)
# print("Users: ", users.shape[0])


query_users = ''' SELECT userId, firstLogin, lastLogin, DATEDIFF(lastLogin, firstLogin)+1 as diff from ML3_mirror.user_data WHERE EXISTS
                  (SELECT userId from log_action WHERE tstamp>='2015-11-04' AND tstamp<'2016-04-01' AND log_action.userId = user_data.userId)
                  AND firstLogin<'2015-12-01' AND firstLogin>='2013-12-01' '''
users_limited = SSH_connection(query_users)
print(users_limited.shape[0])

# users.head()
# users_limited.head()

# 4651
# userId, firstLogin, lastLogin, excludeFromExperiments, isDeleted
# Am I asking this correctly? | Am I asking the right thing?
# choose some qualifications for users that need to be accounted for in this study | there isn't one correct choice -- but think about why a choice could be incorrect

# How many switched recommenders at least once?
# Total number of recommender change events

connection done
6697


In [14]:
users_limited.head()

,userId,firstLogin,lastLogin,diff
0,244830,2013-12-03,2016-03-10,829
1,244847,2013-12-03,2020-10-11,2505
2,244860,2013-12-04,2022-03-11,3020
3,244955,2013-12-08,2021-08-16,2809
4,244940,2013-12-08,2022-05-15,3081


In [19]:
# users = users[users['diff'] >= 60]

# temp1 = users_limited
# temp2 = users

# start = time.time()
# users_limited = users_limited[users_limited['diff'] > 100]
# print("time: ", time.time()-start)

# print(users.shape[0])
# print(users_limited.shape[0])
# print(temp1.shape[0])

time:  0.00279998779296875
9076
3427
3431


In [3]:
BASELINE = "{\"engineId\":\"baseline\"}"
ITEM_ITEM = "{\"engineId\":\"item-item\"}"
SVD = "{\"engineId\":\"svd\"}"
PICK_GROUPS = "{\"engineId\":\"pick-groups\"}"

LONG_TO_SHORT = {BASELINE: "baseline", ITEM_ITEM: "item-item", SVD:"svd", PICK_GROUPS: "pick-groups"}
SHORT_TO_LONG = {"baseline": BASELINE, "item-item":ITEM_ITEM, "svd":SVD, "pick-groups":PICK_GROUPS}

In [37]:
def primary_dicts(users, change_events, fields):
    
    uid = fields[0]
    initial_val = fields[1]
    
    # pattern_list_dict = {} # dict to store key=userId, val=curralg in the iteration process
    
    # how would I know what the initial algorithm assignment is?
    user_id_keys = users[uid].to_list()
    pattern_list_dict = dict.fromkeys(user_id_keys, [ SHORT_TO_LONG[initial_val] ])
    # print("Users: ", len(pattern_list_dict))

    changes_dict = {} # needed to later calculate median changes per user
    # changes_count = change_events.shape[0]
    cmp_field = fields[2]
    # loop to count number of changes correctly; comparing next with prev algorithm
    for index,row in change_events.iterrows():
        userId = row[uid]
        item_to_cmp = row[cmp_field]
        if(pattern_list_dict[userId][-1] != item_to_cmp):
            changes_dict[userId] = 1 + changes_dict.get(userId, 0)
            pattern_list_dict[userId].append(item_to_cmp)

    return (changes_dict, pattern_list_dict)

In [6]:
query_changes = ''' SELECT userId, action, tstamp, logJson from ML3_mirror.log_action WHERE EXISTS
                  (SELECT userId from user_data WHERE firstLogin<'2015-12-01' AND firstLogin>'2013-12-01' AND user_data.userId = log_action.userId)
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01'  AND action='recommender-change' '''
change_events = SSH_connection(query_changes)
print("Rec changes: ", change_events.shape[0])

connection done
Rec changes:  14030


In [19]:
query_changes_days = ''' SELECT userId, action, tstamp, logJson from ML3_mirror.log_action WHERE EXISTS
                  (SELECT userId, DATEDIFF(lastLogin, firstLogin)+1 as diff from user_data WHERE firstLogin<'2015-12-01' 
                  AND firstLogin>'2013-12-01' AND user_data.userId = log_action.userId
                  HAVING diff>=100)
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01'  AND action='recommender-change' '''
change_events_days = SSH_connection(query_changes_days)
print("Rec changes days: ", change_events_days.shape[0])

connection done
Rec changes days:  7480


In [50]:
#TEST
for index,row in change_events.iterrows():
    if row['userId'] == 271900:
        print(row)

userId                       271900
action           recommender-change
tstamp          2015-11-04 00:02:21
logJson    {"engineId":"item-item"}
Name: 1, dtype: object
userId                         271900
action             recommender-change
tstamp            2015-11-04 00:02:49
logJson    {"engineId":"pick-groups"}
Name: 2, dtype: object
userId                         271900
action             recommender-change
tstamp            2015-11-04 00:02:49
logJson    {"engineId":"pick-groups"}
Name: 3, dtype: object
userId                       271900
action           recommender-change
tstamp          2015-11-04 00:03:38
logJson    {"engineId":"item-item"}
Name: 4, dtype: object


In [9]:
query_first_logs = ''' SELECT userId, action, MIN(tstamp) as tstamp from ML3_mirror.log_action WHERE EXISTS
                  (SELECT userId from user_data WHERE firstLogin<'2015-12-01' AND firstLogin>='2013-12-01' AND user_data.userId = log_action.userId)
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01' GROUP BY log_action.userId '''
first_logs = SSH_connection(query_first_logs)
print(first_logs.shape[0])
first_logs.head()

#9m 50.5s

connection done
6697


,userId,action,tstamp
0,244830,pageview,2015-11-04 00:35:31
1,244847,pageview,2015-11-17 13:54:33
2,244860,pageview,2016-03-07 14:36:31
3,244940,pageview,2015-11-08 05:00:12
4,244955,pageview,2016-01-12 22:38:33


In [6]:
#TEST
query = ''' SELECT userId, DATEDIFF(lastLogin, firstLogin)+1 as diff from user_data WHERE firstLogin<'2015-12-01' AND
            firstLogin>='2013-12-01' HAVING diff>=100 '''
df = SSH_connection(query)
df.head()

connection done


,userId,diff
0,244777,180
1,244815,481
2,244817,3148
3,244838,153
4,244841,377


In [5]:
query_logs_days = ''' SELECT userId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
                  (SELECT userId, DATEDIFF(lastLogin, firstLogin)+1 as diff from user_data WHERE firstLogin<'2015-12-01' AND 
                  firstLogin>='2013-12-01' AND log_action.userId=user_data.userId HAVING diff>=100)
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01' '''
user_logs_days = SSH_connection(query_logs_days)
user_logs_days.head()

# 2m 34.3s

connection done


,userId,action,tstamp
0,271892,pageview,2015-11-04 00:00:12
1,271892,pageview,2015-11-04 00:00:22
2,271454,pageview,2015-11-04 00:00:28
3,271892,pageview,2015-11-04 00:00:39
4,271454,pageview,2015-11-04 00:00:45


In [16]:
query_flogs_days = ''' SELECT userId, action, MIN(tstamp) as tstamp from ML3_mirror.log_action WHERE EXISTS 
                  (SELECT userId, DATEDIFF(lastLogin, firstLogin)+1 as diff from user_data WHERE firstLogin<'2015-12-01' AND 
                  firstLogin>='2013-12-01' AND log_action.userId=user_data.userId HAVING diff>=100)
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01' GROUP BY userId '''
first_logs_days = SSH_connection(query_flogs_days)
first_logs_days.shape[0]
first_logs_days.head()

connection done


,userId,action,tstamp
0,244830,pageview,2015-11-04 00:35:31
1,244847,pageview,2015-11-17 13:54:33
2,244860,pageview,2016-03-07 14:36:31
3,244940,pageview,2015-11-08 05:00:12
4,244955,pageview,2016-01-12 22:38:33


In [17]:
print(first_logs_days.shape[0])

3431


In [7]:
query_logs_sessions = ''' SELECT userId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
                  (SELECT userId from user_data WHERE firstLogin<'2015-12-01' AND firstLogin>='2013-12-01' AND log_action.userId=user_data.userId) 
                  AND tstamp>='2015-11-04' AND tstamp<'2016-04-01' '''
user_logs_sessions = SSH_connection(query_logs_sessions)
user_logs_sessions.head()

#2m 54.4s

connection done


,userId,action,tstamp
0,271897,rating,2015-11-04 00:00:00
1,271464,pageview,2015-11-04 00:00:00
2,271869,rating,2015-11-04 00:00:02
3,271897,pageview,2015-11-04 00:00:02
4,271464,rating,2015-11-04 00:00:05


In [10]:
first_logs_tstamps = first_logs.set_index('userId').to_dict()['tstamp']
print(type(user_logs_sessions['userId'].to_list()))

<class 'list'>


In [14]:
print("Days:", user_logs_days.shape[0])
print("Sessions: ", user_logs_sessions.shape[0])

Days: 908779
Sessions:  1210234


In [11]:
def session_count(user_logs, tstamps_dict, fields):
    id_field = fields[0]
    time_field = fields[1]

    sessions_dict = {}

    for index,row in user_logs.iterrows():
        unique_id = row[id_field]
        diff = row[time_field] - tstamps_dict[unique_id]
        if diff.total_seconds() > 3600:
            sessions_dict[unique_id] = 1 + sessions_dict.get(unique_id, 1)
        else:
            sessions_dict[unique_id] = sessions_dict.get(unique_id, 1)
        tstamps_dict[unique_id] = row[time_field]
    
    return sessions_dict

In [15]:
sessions_count = session_count(user_logs_sessions, first_logs_tstamps, ('userId', 'tstamp'))
sessions_count_new = {key:val for key,val in sessions_count.items() if val>=2}
print(max(sessions_count_new.values()))
print(min(sessions_count_new.values()))
print("Users who had a minimum of two sessions between 2015-11-04 and 2016-04-01: ",len(sessions_count_new.values()))
print("Users who used the system between 2015-11-04 and 2016-04-01 and their lastLogin-firstLogin is in range: ",len(sessions_count.values()))

512
2
Users who had a minimum of two sessions between 2015-11-04 and 2016-04-01:  3346
Users who used the system between 2015-11-04 and 2016-04-01 and their lastLogin-firstLogin is in range:  6697


In [41]:
dicts = primary_dicts(first_logs, change_events, ('userId', "pick-groups", 'logJson'))
dict1 = dicts[0]

print("firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action ")
print("Users: ", len(sessions_count))
print("All users who have switched recommenders: ", len(dict1.values()), "(",(len(dict1.values())/6697)*100, "% )")
print("Total rec change events for the same users: ", sum(dict1.values()))
print("")

dict1 = {key:val for key,val in dict1.items() if key in sessions_count_new.keys()}
print("firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action; minimum of two sessions in the time period of interest ")
print("Users: ", len(sessions_count_new))
print("Users of interest who have switched recommenders: ", len(dict1.values()),"(",(len(dict1.values())/3346)*100, "% )")
print("Total rec change events for these users: ",sum(dict1.values()))


firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action 
Users:  6697
All users who have switched recommenders:  3269 ( 48.81290129908915 % )
Total rec change events for the same users:  8556

firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action; minimum of two sessions in the time period of interest 
Users:  3346
Users of interest who have switched recommenders:  1697 ( 50.717274357441724 % )
Total rec change events for these users:  6122


In [45]:
dicts_days = primary_dicts(first_logs_days, change_events_days, ('userId', "pick-groups", 'logJson'))
dict1_days = dicts_days[0]

print("firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action; used the system for at least 100 days ")
print("Users: ", first_logs_days.shape[0])
print("All users who have switched recommenders: ", len(dict1_days.values()), "(",(len(dict1_days.values())/3431)*100, "% )")
print("Total rec change events for the same users: ", sum(dict1_days.values()))

# dict1 = {key:val for key,val in dict1.items() if val!=0}
# print("All users who switched at least once: ", len(dict1.values()))

# have I possibly found evidence that leans towards why picking users who used the system for 100 days or more could mean that 
# they could've performed many actions, including recommender changes, outside of the time period of interest (2015/11 - 2016/03)?

# are these results useful in the context of analyzing recommender changes within the time period of interest?

# defining a group of users who are active | using ratings/ those who logged in a certain number of times within the observation period
# is the minimum number really arbitrary? | sensitivity analysis

firstLogin is between 2013 and 2015; have entries between 2015/11 and 2016/03 in log_action; used the system for at least 100 days 
Users:  3431
All users who have switched recommenders:  1237 ( 36.053628679685225 % )
Total rec change events for the same users:  4591


In [5]:
# SETTING UP DICTS 
# start = time.time()
# temp3 = temp1
# users_df = remove_rows(temp1, 100)
# print("time: ", time.time() - start)

# temp_var = users_limited
# users_limited = users_limited[users_limited['diff'] > 100]
# print(users_limited.shape[0])
# print(temp_var.shape[0])
# print(temp3.shape[0])
# dicts = primary_dicts(users_df, change_events, ('userId', 'group', 'logJson'))
# changes_dict = dicts[0]
# recchange_dict = dicts[1]


# # TABLE 1
# print("Recommender change events: ", sum(changes_dict.values()))
# changes_dict = {key:val for key,val in changes_dict.items() if val !=0}
# print("Users switching at least once: ", len(changes_dict.values()))
# print("Median changes per user w/ at least 1 change: ", int(np.median(list(changes_dict.values()))))

3427
6697
